PSN

In [ ]:

from psnawp_api import PSNAWP
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
from psnawp_api.models import SearchDomain

load_dotenv()

NPSSO = os.getenv("NPSSO")
# Get your npsso from https://psnprofiles.com/psnawp
npsso = NPSSO
# Create a PSNAWP object
psn = PSNAWP(npsso)

# Profile of npsso owner
client = psn.me()
print("account_Id : " + client.account_id + ", online_id : " + client.online_id)
print("totTrophies : " + str(client.trophy_summary().earned_trophies.bronze + client.trophy_summary().earned_trophies.silver + client.trophy_summary().earned_trophies.gold + client.trophy_summary().earned_trophies.platinum))
print("totGames : " + str(list(client.title_stats()).__len__()))

listOfListGames = []
listOfListTrophy = []

def get_np_communication_id(title_id):
    try:
        game_title = psn.game_title(title_id=title_id, account_id="me")
        product_id = game_title.get_details()[0].get("id", None)
        return {
            "np_communication_id": game_title.np_communication_id,
            "product_id": product_id,
        }
    except Exception as e:
        print(f"Error retrieving np_communication_id for title_id {title_id}: {e}")
        return {"np_communication_id": None, "product_id": None}

def get_product_id_by_title(title_name):
    try:
        search = psn.search(search_query=title_name, search_domain=SearchDomain.FULL_GAMES)
        for i in search:
            product_id = i.get("id", None)
            break
        return product_id if product_id else None
    except Exception as e:
        # print(f"Error retrieving product_id for game title {title_name}: {e}")
        return None
    
# title_stats = list(client.title_stats())
# trophy_stats = list(client.trophy_titles())

gameCount = 0
for t in tqdm(client.title_stats(), desc="Processing games"):
    print("name : " + t.name + " , title_id : " + str(t.title_id))
    ids = get_np_communication_id(t.title_id)
    np_communication_id = ids["np_communication_id"]
    product_id = ids["product_id"]
    print("np_communication_id : " + str(np_communication_id) + " , product_id : " + str(product_id))
    listGame = [
        t.title_id,
        t.name,
        t.image_url,
        t.category,
        t.play_count,
        t.first_played_date_time,
        t.last_played_date_time,
        t.play_duration,
        np_communication_id,
        product_id]
    listOfListGames.append(listGame)
    gameCount += 1

np_communication_id_list = [game[8] for game in listOfListGames if game[8] is not None]

earnedTrophyCount = 0
totTrophyCount = 0
completeTrophyCount = 0
for tr in tqdm(client.trophy_titles(), desc="Processing trophies"):
    if tr.np_communication_id not in np_communication_id_list:
        product_id = get_product_id_by_title(tr.title_name)
    else:
        product_id = next((game[9] for game in listOfListGames if game[8] == tr.np_communication_id), None)
    listGame = [tr.np_communication_id, tr.title_name,
                (tr.defined_trophies.bronze + tr.defined_trophies.silver + tr.defined_trophies.gold + tr.defined_trophies.platinum),
                (tr.earned_trophies.bronze + tr.earned_trophies.silver + tr.earned_trophies.gold + tr.earned_trophies.platinum),
                (str(tr.progress) + "%"), tr.np_title_id, product_id]
    listOfListTrophy.append(listGame)
    earnedTrophyCount += (tr.earned_trophies.bronze + tr.earned_trophies.silver + tr.earned_trophies.gold + tr.earned_trophies.platinum)
    totTrophyCount += (tr.defined_trophies.bronze + tr.defined_trophies.silver + tr.defined_trophies.gold + tr.defined_trophies.platinum)
    if tr.earned_trophies.platinum > 0:
        completeTrophyCount += 1

df_trophy_psn = pd.DataFrame(np.array(listOfListTrophy), columns=["np_communication_id", "title_name", "totTrophy", "earnedTrophy", "percTrophy", "np_title_id", "product_id"])
df_games_psn = pd.DataFrame(np.array(listOfListGames), columns=["title_id", "name", "image_url", "category", "play_count", "first_played_date_time", "last_played_date_time", "play_duration", "np_communication_id", "product_id"])
#In caso fare join tra le due tabelle per ottenere la lista completa di giopchi e dove non c'e la category PS5 o PS4 mettere Ps3

print("Game Count : " + str(gameCount) + ", Earned Trophy Count : " + str(earnedTrophyCount) + ", Total Trophy Count : " + str(totTrophyCount) + ", Complete Trophy Count : " + str(completeTrophyCount))

df_games_psn


In [7]:
df_games_psn = df_games_psn.dropna(subset=["np_communication_id"])
df_games_psn

,title_id,name,image_url,category,play_count,first_played_date_time,last_played_date_time,play_duration,np_communication_id
0,CUSA38355_00,Bendy and the Dark Revival,https://image.api.playstation.com/vulcan/ap/rn...,PlatformCategory.PS4,3,2025-05-29 14:14:06.040000+00:00,2025-06-02 00:06:41.300000+00:00,0 days 02:06:46,NPWR34357_00
1,CUSA12085_00,Spyro™ Reignited Trilogy,https://image.api.playstation.com/gs2-sec/appk...,PlatformCategory.PS4,23,2021-09-30 19:01:42.120000+00:00,2025-06-01 23:20:09.490000+00:00,0 days 17:04:39,NPWR15891_00
2,PPSA25381_00,ELDEN RING NIGHTREIGN,https://image.api.playstation.com/vulcan/ap/rn...,PlatformCategory.PS5,6,2025-05-30 18:20:12.180000+00:00,2025-06-01 18:26:34.460000+00:00,0 days 08:32:53,NPWR44031_00
3,PPSA08709_00,SILENT HILL 2,https://image.api.playstation.com/vulcan/ap/rn...,PlatformCategory.PS5,33,2025-05-18 16:28:59.410000+00:00,2025-06-01 12:39:18.080000+00:00,1 days 07:12:57,NPWR30769_00
4,CUSA14876_00,Crash™ Team Racing Nitro-Fueled,https://image.api.playstation.com/gs2-sec/appk...,PlatformCategory.PS4,10,2021-09-30 09:05:33.710000+00:00,2025-05-31 17:06:33.870000+00:00,0 days 04:10:30,NPWR16560_00
...,...,...,...,...,...,...,...,...,...
610,CUSA00293_00,The Swapper,https://image.api.playstation.com/gs2-sec/appk...,PlatformCategory.PS4,2,2015-02-08 18:45:21+00:00,2015-02-08 18:45:21+00:00,0 days 01:12:34,NPWR05841_00
611,CUSA00105_00,Pinball Arcade,https://image.api.playstation.com/gs2-sec/appk...,PlatformCategory.PS4,4,2014-12-31 14:56:20+00:00,2015-01-01 18:14:06+00:00,0 days 00:16:10,NPWR05356_00
612,CUSA00186_00,Zen Pinball 2,https://image.api.playstation.com/gs2-sec/appk...,PlatformCategory.PS4,8,2014-12-31 14:58:43+00:00,2015-01-01 18:04:03+00:00,0 days 00:55:52,NPWR05611_00
613,CUSA00932_00,Invokers Tournament™,https://image.api.playstation.com/gs2-sec/appk...,PlatformCategory.UNKNOWN,4,2014-12-31 15:43:36+00:00,2014-12-31 17:50:13+00:00,0 days 00:51:58,NPWR06502_00


In [67]:
df_trophy_psn

,np_communication_id,title_name,totTrophy,earnedTrophy,percTrophy,np_title_id,product_id
0,NPWR39144_00,Clair Obscur: Expedition 33,56,10,14%,None,None
1,NPWR30769_00,SILENT HILL 2,44,26,40%,None,None
2,NPWR15892_00,Spyro 3: Year of the Dragon,41,7,20%,None,None
3,NPWR37403_00,High on Life,45,11,13%,None,None
4,NPWR15891_00,Spyro 2: Ripto's Rage!,30,14,53%,None,None
...,...,...,...,...,...,...,...
650,NPWR00518_00,The Godfather™ II,42,35,72%,None,None
651,NPWR00650_00,L.A. Noire,61,12,10%,None,None
652,NPWR01768_00,SONIC GENERATIONS,50,26,39%,None,None
653,NPWR01500_00,DCUO Trophies,123,0,0%,None,None


In [ ]:
#Si fa il merge in maniera right per includere anche i giochi PS3 da riempirne i dati poi tramite metadata
df_merged = pd.merge(df_games_psn, df_trophy_psn, on="np_communication_id", how="right")
df_merged

,title_id,name,image_url,category,play_count,first_played_date_time,last_played_date_time,play_duration,np_communication_id,title_name,totTrophy,earnedTrophy,percTrophy,np_title_id
0,PPSA15389_00,High On Life,https://image.api.playstation.com/vulcan/ap/rn...,PlatformCategory.PS5,4,2024-06-16 19:44:39.180000+00:00,2024-06-18 10:16:12.890000+00:00,0 days 05:19:42,NPWR37403_00,High on Life,45,8,9%,None
1,CUSA38355_00,Bendy and the Dark Revival,https://image.api.playstation.com/vulcan/ap/rn...,PlatformCategory.PS4,3,2025-05-29 14:14:06.040000+00:00,2025-06-02 00:06:41.300000+00:00,0 days 02:06:46,NPWR34357_00,Bendy and the Dark Revival,36,5,8%,None
2,CUSA12085_00,Spyro™ Reignited Trilogy,https://image.api.playstation.com/gs2-sec/appk...,PlatformCategory.PS4,23,2021-09-30 19:01:42.120000+00:00,2025-06-01 23:20:09.490000+00:00,0 days 17:04:39,NPWR15891_00,Spyro 2: Ripto's Rage!,30,7,30%,None
3,PPSA25381_00,ELDEN RING NIGHTREIGN,https://image.api.playstation.com/vulcan/ap/rn...,PlatformCategory.PS5,6,2025-05-30 18:20:12.180000+00:00,2025-06-01 18:26:34.460000+00:00,0 days 08:32:53,NPWR44031_00,ELDEN RING NIGHTREIGN,37,7,11%,None
4,PPSA08709_00,SILENT HILL 2,https://image.api.playstation.com/vulcan/ap/rn...,PlatformCategory.PS5,33,2025-05-18 16:28:59.410000+00:00,2025-06-01 12:39:18.080000+00:00,1 days 07:12:57,NPWR30769_00,SILENT HILL 2,44,24,37%,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NPWR00518_00,The Godfather™ II,42,35,72%,None
654,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NPWR00650_00,L.A. Noire,61,12,10%,None
655,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NPWR01768_00,SONIC GENERATIONS,50,26,39%,None
656,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NPWR01500_00,DCUO Trophies,123,0,0%,None


Steam

In [2]:
from steam_web_api import Steam
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
from dotenv import load_dotenv

load_dotenv()

KEY = os.getenv("STEAM_API_KEY")
steam = Steam(KEY)

user = steam.users.get_user_details("76561198074617013")
# print(user)
# print(user["player"])
# print("PLAYER DETAILS")
# for i in user["player"]:
# print(str(i) + " : " + str(user["player"][i]))

print(
    "STEAMID: "
    + str(user["player"]["steamid"])
    + " NAME : "
    + str(user["player"]["personaname"])
)


# friends = steam.users.get_user_friends_list("76561198074617013")
# print(friends)

steamId = str(user["player"]["steamid"])  # 76561198074617013

listOfList = []


# GAMES
games = steam.users.get_owned_games(steamId)
print("Game Count : " + str(games.get("game_count")))
print("Recupero Dati : ")
for i in tqdm(games.get("games")):
    listGame = [i.get("appid"), i.get("name"), i.get("playtime_forever")]
    # print("appId: " + str(i.get("appid")) + " / Name: " + str(i.get("name")) + " / Playtime: " + str(i.get("playtime_forever")))
    try:
        if i.get("playtime_forever") > 0:
            gameS = steam.apps.get_user_stats(steamId, i.get("appid"))
            achievements = steam.apps.get_user_achievements(steamId, i.get("appid"))
            g = steam.apps.get_app_details(i.get("appid"))
            totAchievement = int(
                len(achievements.get("playerstats").get("achievements"))
            )
            earnedAchievement = len(gameS.get("playerstats").get("achievements"))
            # print("Tot Achievement : " + str(totAchievement))
            listGame.append(str(totAchievement))
            # print("Earned Achievement : " + str(earnedAchievement))
            listGame.append(str(earnedAchievement))
            if totAchievement > 0:
                percAchievement = (earnedAchievement / totAchievement) * 100
                # print("Achievement % : " + str(round(percAchievement, 2)) + "%")
                listGame.append(str(round(percAchievement, 2)) + "%")
            else:
                # print("Achievement % : 0 %")
                listGame.append(str(0) + "%")
        else:
            # print("No Achievement Data")
            listGame.append(str(0))
            listGame.append(str(0))
            listGame.append(str(0) + "%")
    except:
        # print("No Achievement Data")
        listGame.append(str(0))
        listGame.append(str(0))
        listGame.append(str(0) + "%")
    listOfList.append(listGame)

# TODO : organizzare dati in dataFrame

arr = np.array(listOfList)
df_games_steam = pd.DataFrame(
    arr,
    columns=[
        "appId",
        "Name",
        "Playtime",
        "TotAchievement",
        "EarnedAchievement",
        "Achievement%",
    ],
)


searchGame = steam.apps.search_games("Dota 2")
# print(searchGame)

gameAchievement = steam.apps.get_user_achievements("76561198074617013", 220)
# print(gameAchievement)

gameStats = steam.apps.get_user_stats("76561198074617013", 220)
# print(gameStats)

gameDetail = steam.apps.get_app_details(220)
# print(gameDetail)

df_games_steam


STEAMID: 76561198074617013 NAME : AterMors
Game Count : 286
Recupero Dati : 


100%|██████████| 286/286 [02:59<00:00,  1.59it/s]


,appId,Name,Playtime,TotAchievement,EarnedAchievement,Achievement%
0,220,Half-Life 2,266,69,4,5.8%
1,320,Half-Life 2: Deathmatch,0,0,0,0%
2,360,Half-Life Deathmatch: Source,0,0,0,0%
3,4000,Garry's Mod,5,29,1,3.45%
4,6910,Deus Ex: Game of the Year Edition,22,0,0,0%
...,...,...,...,...,...,...
281,383180,Dead Island Riptide Definitive Edition,0,0,0,0%
282,235900,RPG Maker XP,0,0,0,0%
283,440,Team Fortress 2,176,520,12,2.31%
284,570,Dota 2,117,0,0,0%


GameDB

In [ ]:
import os
import sys
parent_dir = os.path.dirname(os.getcwd())
wrapper_path = os.path.join(os.path.dirname(os.getcwd()), "RAWG_API_wrapper")
sys.path.append(wrapper_path)
from rawg_client import RAWGClient
from dotenv import load_dotenv

load_dotenv()

KEY = os.getenv("RAWG_API_KEY")


client = RAWGClient(api_key=KEY) #API Key from https://rawg.io/apidocs

results = client.search_games("Super Mario Odyssey")
game_id = results[0]["id"]
metadata = client.get_game_metadata(game_id)

print("TITOLO:", metadata["name"])
print("DATA USCITA:", metadata["released"])
print("PIATTAFORME:", metadata["platforms"])
print("GENERI:", metadata["genres"])
print("SVILUPPATORI:", metadata["developers"])
print("PUBLISHER:", metadata["publishers"])
print("IMMAGINE:", metadata["cover_image"])
print("DESCRIZIONE:", metadata["description"][:500])

IGDB

In [ ]:
from igdb.wrapper import IGDBWrapper
from igdb.igdbapi_pb2 import GameResult
import json
import pandas as pd


wrapper = IGDBWrapper(
    "", ""
)

# # JSON API request
# byte_array = wrapper.api_request(
#     "games", "fields id, name; offset 0; where platforms=48; limit 1;"
# )
# # parse into JSON however you like...

# # Protobuf API request

# byte_array = wrapper.api_request(
#     "games.pb",  # Note the '.pb' suffix at the endpoint
#     "fields id, name; offset 0; where platforms=48; limit 1;",
# )
# games_message = GameResult()
# games_message.ParseFromString(
#     byte_array
# )  # Fills the protobuf message object with the response
# games = games_message.games

# games

# Game name you want to search
game_name = "Persona 5"

# where name ~ *"{game_name}"* & category = (0, 1, 2, 4, 6, 7, 8, 9, 11);


# Query string (search "{game_name}";)
query = f'''
fields 
  name,
  summary,
  storyline,
  genres.name,
  themes.name,
  keywords.name,
  platforms.name,
  release_dates.date,
  release_dates.human,
  release_dates.platform.name,
  first_release_date,
  involved_companies.company.name,
  involved_companies.developer,
  involved_companies.publisher,
  involved_companies.porting,
  involved_companies.supporting,
  cover.url,
  artworks.url,
  screenshots.url,
  videos.video_id,
  websites.url,
  websites.category,
  game_modes.name,
  player_perspectives.name,
  category,
  status,
  total_rating,
  total_rating_count,
  aggregated_rating,
  aggregated_rating_count,
  collection.name,
  franchise.name,
  similar_games.name;

limit 20;
'''

# Send request to 'games' endpoint
result = wrapper.api_request("games", query)
result = json.loads(result)
result_df = pd.DataFrame(result)
# Print formatted result
# print(json.dumps(result, indent=2))
result_df

,id,category,cover,first_release_date,game_modes,genres,involved_companies,name,platforms,player_perspectives,...,websites,keywords,storyline,videos,artworks,aggregated_rating,aggregated_rating_count,total_rating,total_rating_count,status
0,41866,0,"{'id': 125521, 'url': '//images.igdb.com/igdb/...",1.491264e+09,"[{'id': 1, 'name': 'Single player'}]","[{'id': 12, 'name': 'Role-playing (RPG)'}]","[{'id': 117662, 'company': {'id': 818, 'name':...",Persona 5: Take Your Heart - Premium Edition,"[{'id': 48, 'name': 'PlayStation 4'}]","[{'id': 2, 'name': 'Third person'}]",...,"[{'id': 216549, 'category': 1, 'url': 'http://...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,74879,0,"{'id': 86124, 'url': '//images.igdb.com/igdb/i...",1.473898e+09,"[{'id': 1, 'name': 'Single player'}]","[{'id': 8, 'name': 'Platform'}, {'id': 12, 'na...","[{'id': 87848, 'company': {'id': 818, 'name': ...",Persona 5: Ultimate Edition,"[{'id': 9, 'name': 'PlayStation 3'}, {'id': 48...","[{'id': 2, 'name': 'Third person'}]",...,"[{'id': 125954, 'category': 5, 'url': 'https:/...","[{'id': 78, 'name': 'anime'}, {'id': 98, 'name...",The story focuses on the 16-year-old protagoni...,"[{'id': 31519, 'video_id': 'QnDzJ9KzuV4'}, {'i...",NaN,NaN,NaN,NaN,NaN,NaN
2,242315,0,"{'id': 457688, 'url': '//images.igdb.com/igdb/...",1.712880e+09,"[{'id': 5, 'name': 'Massively Multiplayer Onli...","[{'id': 12, 'name': 'Role-playing (RPG)'}, {'i...","[{'id': 226478, 'company': {'id': 112, 'name':...",Persona 5: The Phantom X,"[{'id': 34, 'name': 'Android'}, {'id': 6, 'nam...","[{'id': 2, 'name': 'Third person'}]",...,"[{'id': 791053, 'category': 13, 'url': 'https:...",NaN,NaN,"[{'id': 86492, 'video_id': 'XhqpSJiRzUs'}, {'i...","[{'id': 167430, 'url': '//images.igdb.com/igdb...",NaN,NaN,NaN,NaN,NaN
3,261539,0,"{'id': 337625, 'url': '//images.igdb.com/igdb/...",1.694736e+09,NaN,NaN,NaN,Persona 5 Royal: 1 More Edition,"[{'id': 169, 'name': 'Xbox Series X|S'}, {'id'...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9927,0,"{'id': 81906, 'url': '//images.igdb.com/igdb/i...",1.473898e+09,"[{'id': 1, 'name': 'Single player'}]","[{'id': 12, 'name': 'Role-playing (RPG)'}, {'i...","[{'id': 23554, 'company': {'id': 818, 'name': ...",Persona 5,"[{'id': 9, 'name': 'PlayStation 3'}, {'id': 48...","[{'id': 2, 'name': 'Third person'}]",...,"[{'id': 783437, 'url': 'https://store.playstat...","[{'id': 78, 'name': 'anime'}, {'id': 175, 'nam...",The story focuses on the 16-year-old protagoni...,"[{'id': 11727, 'video_id': 'w9MAhxxrNFk'}, {'i...","[{'id': 3797, 'url': '//images.igdb.com/igdb/i...",93.571429,21.0,93.173424,947.0,NaN
5,117731,0,"{'id': 139562, 'url': '//images.igdb.com/igdb/...",1.582157e+09,"[{'id': 1, 'name': 'Single player'}]","[{'id': 12, 'name': 'Role-playing (RPG)'}, {'i...","[{'id': 239277, 'company': {'id': 112, 'name':...",Persona 5 Strikers,"[{'id': 48, 'name': 'PlayStation 4'}, {'id': 6...","[{'id': 2, 'name': 'Third person'}]",...,"[{'id': 105166, 'category': 1, 'url': 'https:/...","[{'id': 1075, 'name': 'crossover'}, {'id': 269...",Join the Phantom Thieves and strike back again...,"[{'id': 31147, 'video_id': 'RKEKHvJJ5Mg'}, {'i...","[{'id': 12712, 'url': '//images.igdb.com/igdb/...",81.230769,13.0,80.336717,82.0,NaN
6,256154,0,"{'id': 352084, 'url': '//images.igdb.com/igdb/...",1.700179e+09,NaN,NaN,"[{'id': 245252, 'company': {'id': 818, 'name':...",Persona 5 Tactica: Launch Edition,"[{'id': 169, 'name': 'Xbox Series X|S'}, {'id'...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,167115,0,"{'id': 337815, 'url': '//images.igdb.com/igdb/...",1.491264e+09,NaN,"[{'id': 12, 'name': 'Role-playing (RPG)'}]","[{'id': 154550, 'company': {'id': 2488, 'name'...",Persona 5: Steelbook Edition,"[{'id': 48, 'name': 'PlayStation 4'}]",NaN,...,"[{'id': 218094, 'category': 1, 'url': 'http://...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,321602,0,"{'id': 443249, 'url': '//images.igdb.com/igdb/...",NaN,NaN,"[{'id': 12, 'name': 'Role-playing (RPG)'}, {'i...","[{'id': 296034, 'company': {'id': 59

In [32]:
# #Only take url from artworks
# if "artworks" in result[0]:
#     result[0]["artworks"] = [artwork["url"] for artwork in result[0]["artworks"]]
print(result[0].get("release_dates", [])[0].get("human", "N/A"))


Oct 27, 2017


In [ ]:
from igdb.wrapper import IGDBWrapper
from igdb.igdbapi_pb2 import GameResult
import json
import pandas as pd


wrapper = IGDBWrapper(
    "", ""
)

# # JSON API request
# byte_array = wrapper.api_request(
#     "games", "fields id, name; offset 0; where platforms=48; limit 1;"
# )
# # parse into JSON however you like...

# # Protobuf API request

# byte_array = wrapper.api_request(
#     "games.pb",  # Note the '.pb' suffix at the endpoint
#     "fields id, name; offset 0; where platforms=48; limit 1;",
# )
# games_message = GameResult()
# games_message.ParseFromString(
#     byte_array
# )  # Fills the protobuf message object with the response
# games = games_message.games

# games

# Game name you want to search
game_name = "Persona 5"

# where name ~ *"{game_name}"* & category = (0, 1, 2, 4, 6, 7, 8, 9, 11);


# Query string (search "{game_name}";)
query = """
fields game.name, uid, category;
where uid = "10008503" & category = (1, 36);  // 220 is Steam ID for Half-Life 2
"""
# Send request to 'games' endpoint
result = wrapper.api_request("external_games", query)
result = json.loads(result)
result_df = pd.DataFrame(result)
# Print formatted result
# print(json.dumps(result, indent=2))
result_df


,id,category,game,uid
0,2978040,36,"{'id': 305152, 'name': 'Clair Obscur: Expediti...",10008503
